In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json
WAIT = 3
#options = webdriver.ChromeOptions()
#options.add_argument('headless')

In [3]:
# css 찾을때 까지 10초대기
def time_wait(num, code,driver):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [4]:
def find_search_input(key_word,driver):
    # 검색창 찾기
    driver.switch_to.default_content()
    time_wait(2, 'div.input_box > input.input_search',driver)
    search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
    search.send_keys(key_word)  # 검색어 입력
    driver.implicitly_wait(3)
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    sleep(5)
    res = driver.page_source  # 페이지 소스 가져오기
    soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다
    sleep(1)

In [5]:
# frame 변경 메소드
def switch_frame(frame,driver):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    driver.implicitly_wait(1)


In [6]:
# 페이지 다운
def page_down(num,driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [7]:
# 페이지 다운
def page_down(num,driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [8]:
def find_review_btn(driver):
    sleep(2)
    try:
        menu_list = driver.find_elements(By.CSS_SELECTOR,'a.tpj9w')
        if len(menu_list) ==7:
            return menu_list[-1]
        else:
            return menu_list[-2]
    except:
        print('리뷰가 존재하지 않습니다.')

# 메뉴&리뷰 크롤링 함수

In [16]:
def crawling_menu_review(driver):
    #-----리뷰 버튼 클릭-------
    find_review_btn(driver).click()
    SCROLL_PAUSE_SEC = 1
    sleep(1)
    page_down(1,driver)
    driver.implicitly_wait(WAIT)
    #----------------메뉴, 크롤링 블록----------------
    menu_list = list()
    review_list = list()
    try:
        # --------------메뉴 크롤링 블록--------------------------
        try :
            #메뉴 요소 찾기
            menu_elements  = driver.find_elements(By.CSS_SELECTOR,'div.place_fixed_subtab > div > div:nth-child(1) > div > div > div > span > a > span:first-child')
            sleep(SCROLL_PAUSE_SEC)
            driver.implicitly_wait(WAIT)
            try:
                #왼쪽 버튼 찾기
                left_btn = driver.find_element(By.CSS_SELECTOR,'a.PznE8.aNw43')
                driver.implicitly_wait(WAIT)
                left_btn.click()
                driver.implicitly_wait(WAIT)
            except:
                pass

            try:
                #오른족 버튼 찾기
                right_btn = driver.find_element(By.CSS_SELECTOR, 'div.place_fixed_subtab > div > div:nth-child(1) > div > div > a.ZCqf_.aNw43')
                driver.implicitly_wait(WAIT)
            except:
                pass
            #-----------여기까지-----------
            i = 0
            # menu_elentents가 존재하지 않는다면 except로 갈 것이다.
            try: 
                while True:
                    if menu_elements[i].text != '': #요소가 존재하고 메뉴 텍스트도 보일때
                        menu_list.append(menu_elements[i].text)
                        if i == len(menu_elements)-1: # 메뉴 리스트 길이면 나온나
                            break
                        i+=1
                    else:
                        right_btn.click()
                        sleep(SCROLL_PAUSE_SEC)
                        driver.implicitly_wait(WAIT)
            except:
                pass

            #last_scroll = driver.execute_script("return document.body.scrollHeight")
            #view_more_count = 0

        except:
            pass

        # --------------리뷰 크롤링 블록--------------------------
        try:
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                sleep(1)
                driver.implicitly_wait(SCROLL_PAUSE_SEC)

                #if view_more_count ==19:
                    #break
                try :
                    #더보기 버튼 클릭
                    driver.implicitly_wait(WAIT)
                    driver.find_element(By.CSS_SELECTOR,'a.fvwqf').click()
                    #view_more_count += 1

                except:
                    #더보기 버튼이 없다면 while문을 빠져나온다.
                    break
                    # 끝까지 스크롤 다운

                #new_scroll = driver.execute_script("return document.body.scrollHeight")
                #driver.implicitly_wait(WAIT)
                #if last_scroll == new_scroll:
                #    break
                #else:
                #    last_scroll= new_scroll

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            #가게 리스트 마다 방문자 리뷰 가져오기
            review_elements = driver.find_elements(By.CSS_SELECTOR,"span.zPfVt")
            for i in review_elements:
                review_list.append(i.text)
        except:
            pass
    
    except:
        pass
    
    return menu_list, review_list

# 전체 크롤링 함수

In [27]:
def crawling(driver, i, store_elements,review_dic):
    driver.implicitly_wait(1)
    store_elements[i].click() #li의 스토어를 클릭
    driver.implicitly_wait(WAIT) # 2초 휴식
    try:
        #다른 iframe으로 변경
        switch_frame('entryIframe',driver)
        time_wait(2, '.Fc1rA',driver)

        # -----매장명 가져오기-----
        store_name = driver.find_element(By.CSS_SELECTOR,'.Fc1rA').text
        driver.implicitly_wait(2)
        review_dic[store_name] = {}
        
        # -----평점 가져오기-----
        try :
            rank = driver.find_element(By.CSS_SELECTOR,'div.zD5Nm.f7aZ0 > div.dAsGb > span.PXMot.LXIwF > em')
            driver.implicitly_wait(WAIT)
            review_dic[store_name]['rank'] = rank.text
        except:
            pass

        # -----위치 가져오기-----
        try : 
            location = driver.find_element(By.CSS_SELECTOR, 'span.IH7VW').text
            driver.implicitly_wait(WAIT)
            review_dic[store_name]['location'] = location
        except:
            pass
        
        
        #---------------문제--------------
        memu_list, review_list = crawling_menu_review(driver)
        
        # 메뉴 크롤링
        review_dic[store_name]['menu'] = menu_list
        
        # 방문자 리뷰 크롤링
        review_dic[store_name]['review'] = review_list
        driver.implicitly_wait(WAIT)
        
    except:
        pass
    
    memu_list, review_list = crawling_menu_review(driver)
        
    # 메뉴 크롤링
    review_dic[store_name]['menu'] = memu_list

    # 방문자 리뷰 크롤링
    review_dic[store_name]['review'] = review_list
    driver.implicitly_wait(WAIT)
    
    return store_name

# 메인 함수

In [38]:
def main(key_word, page_num):
    review_dic = dict()
    page_num = page_num-1
    refresh_count = 0
    
    url = 'https://map.naver.com/v5/search'
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    driver.implicitly_wait(2)

    # 검색어 입력
    find_search_input(key_word,driver=driver)
    driver.implicitly_wait(2)

    #프레임 변경
    switch_frame('searchIframe',driver)
    driver.implicitly_wait(WAIT)

    # 페이지 버튼 찾기
    pages = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
    if page_num <= 5:
        pages[page_num].click()
        driver.implicitly_wait(WAIT)
    else:
        driver.find_elements(By.CSS_SELECTOR,'div.zRM9F > a:nth-child(6)').click()
        driver.implicitly_wait(WAIT)
        driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
        driver.implicitly_wait(WAIT)

    driver.implicitly_wait(WAIT)
    page_down(40,driver)
    sleep(1)
    driver.implicitly_wait(1)
    page_down(5,driver)
    sleep(2)
    driver.execute_script("window.scrollTo(0, 0);")
    driver.implicitly_wait(WAIT)
    store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
    print('total store_list : ', len(store_elements)) #매장리스트 접근
    # 체크포인트
    ck_pt_idx = 0
    
    while True:
        while True:
            store_name = crawling(driver, ck_pt_idx, store_elements, review_dic)
            ck_pt_idx +=1
            switch_frame('searchIframe',driver)
            
            #한 refresh를 벗어나는 조건문
            if len(review_dic[store_name]['review']) >= 500:
                break
        
        
        driver.refresh()
        switch_frame('searchIframe',driver)
        driver.implicitly_wait(WAIT)
        page_down(40,driver)
        sleep(1)
        driver.implicitly_wait(1)
        page_down(5,driver)
        sleep(2)
        
        store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
        
        # 페이지의 크롤링을 나오는 조건문
        if ck_pt_idx == len(store_elements)-1:
            break
        

    # json 파일로 저장
    with open(f'./Crawling2/naver_{key_word}_review_dic_page{page_num+1}.json', 'w', encoding='utf-8') as f:
        json.dump(review_dic, f, indent=4, ensure_ascii=False)        
    

In [40]:
main('울산 횟집', 1)

C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_14332\316809092.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  44


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C178B3+2193587]
	Ordinal0 [0x00BB0681+1771137]
	Ordinal0 [0x00AC41A8+803240]
	Ordinal0 [0x00AC6BB4+814004]
	Ordinal0 [0x00AC6A72+813682]
	Ordinal0 [0x00AC6D00+814336]
	Ordinal0 [0x00AF3F19+999193]
	Ordinal0 [0x00AE9146+954694]
	Ordinal0 [0x00B0D7AC+1103788]
	Ordinal0 [0x00AE8C04+953348]
	Ordinal0 [0x00B0D9C4+1104324]
	Ordinal0 [0x00B1DAE2+1170146]
	Ordinal0 [0x00B0D5C6+1103302]
	Ordinal0 [0x00AE77E0+948192]
	Ordinal0 [0x00AE86E6+952038]
	GetHandleVerifier [0x00EC0CB2+2738370]
	GetHandleVerifier [0x00EB21B8+2678216]
	GetHandleVerifier [0x00CA17AA+512954]
	GetHandleVerifier [0x00CA0856+509030]
	Ordinal0 [0x00BB743B+1799227]
	Ordinal0 [0x00BBBB68+1817448]
	Ordinal0 [0x00BBBC55+1817685]
	Ordinal0 [0x00BC5230+1856048]
	BaseThreadInitThunk [0x75B8FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77B275F4+228]

In [46]:
review_dic = dict()
key_word = '인천 횟집'
page_num  = 0
refresh_count = 0

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')
driver.get(url)
driver.implicitly_wait(2)

# 검색어 입력
find_search_input(key_word,driver=driver)
driver.implicitly_wait(2)

#프레임 변경
switch_frame('searchIframe',driver)
driver.implicitly_wait(WAIT)

# 페이지 버튼 찾기
pages = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
if page_num <= 5:
    pages[page_num].click()
    driver.implicitly_wait(WAIT)
else:
    driver.find_elements(By.CSS_SELECTOR,'div.zRM9F > a:nth-child(6)').click()
    driver.implicitly_wait(WAIT)
    driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
    driver.implicitly_wait(WAIT)

driver.implicitly_wait(WAIT)
page_down(40,driver)
sleep(1)
driver.implicitly_wait(1)
page_down(5,driver)
sleep(2)
driver.execute_script("window.scrollTo(0, 0);")
driver.implicitly_wait(WAIT)
store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
print('total store_list : ', len(store_elements)) #매장리스트 접근
# 체크포인트

C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_14332\20173889.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  53


In [42]:
switch_frame('searchIframe',driver)

In [47]:
store_name = crawling(driver, 10, store_elements, review_dic)

In [48]:
store_name

'보길도횟집'

In [49]:
review_dic[store_name]

{'rank': '4.34',
 'location': '인천 중구 월미문화로 95',
 'menu': ['게장',
  '매운탕',
  '간장게장',
  '활어회',
  '튀김',
  '꽃게',
  '꽃게찜',
  '새우',
  '회정식',
  '회코스',
  '대하',
  '콘치즈',
  '치즈',
  '초밥',
  '가리비'],
 'review': ['양도 푸짐하고 신선하고 맛있게 잘 먹고 왔답니다~😋👍',
  '맛있고 친절하세요 특별한 날 자주 갑니다',
  '블로그엔 3인 활어셋트 13만원이라 나오던데 걍 15 만원 하나로 통일 됐네유. 알바글이 많아서 블로그 는 그닥 믿음이 안가지만.. 다른집들처럼 쓸데...',
  '가족 5명이 갔어요 스페셜 170,000원짜리를 시켰는 데 나중에 온 손님만 다 챙기구 우리는 개인접시두안 주구 수저두 안주는 거예요.늦게온 테이블이 ...',
  '음…개인적으론 그닥 별 기대는 안했지만 역시는 역시나',
  '와 진짜 푸짐해요... 매운탕이랑 간장게장은 못 먹고 포장했어요,,^_^ 근데 사람이 많아서 정신이 없으셨 나 저희보다 늦게 온 테이블에 회를 먼저 주셔...',
  '활어회코스 좋아요 상차림 정말 푸짐하고, 음식이 끝도없이 나와서 넘넘 행복했음 ㅋㅋ 잔칫상을 방불케하는 비주얼! 잘먹었습니다~',
  '회도 그렇고 음식들이 다채로우면서도 플레이팅도 \n고급스러워서 정성이 가득 느껴지는 곳이었습니다, \n음식에 신경을 많이쓰신것 같아요 덕분에 좋...',
  '간판 커서 찾기 쉽구요! 싱싱한 활어회에다가 푸짐한 스끼다시 즐기고 올수 있었습니다 맛있게 먹었어요 ^^',
  '리뷰가 좋아서 갔는데 후회하고 왔어요 야채도 깨끗\n하게 씻기지도않고 바빠서인지 음식주문하고 한참을\n기다렸는데 기대이하네요 이번에는 리뷰가 틀...',
  '굳',
  '굿',
  '사장님이 인심도 좋으시고 친절하십니다.\n가족 모두 보길도 팬입니다^^',
  '일찍 나가래요',
  '비싸',
  '회 스페샬 주문했는데 이것저것 다양하게많이 나왔

In [50]:
driver.refresh()

In [52]:
crawling(driver, 4, store_elements, review_dic)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C178B3+2193587]
	Ordinal0 [0x00BB0681+1771137]
	Ordinal0 [0x00AC41A8+803240]
	Ordinal0 [0x00AC6BB4+814004]
	Ordinal0 [0x00AC6A72+813682]
	Ordinal0 [0x00AC6D00+814336]
	Ordinal0 [0x00AF3F19+999193]
	Ordinal0 [0x00AE9146+954694]
	Ordinal0 [0x00B0D7AC+1103788]
	Ordinal0 [0x00AE8C04+953348]
	Ordinal0 [0x00B0D9C4+1104324]
	Ordinal0 [0x00B1DAE2+1170146]
	Ordinal0 [0x00B0D5C6+1103302]
	Ordinal0 [0x00AE77E0+948192]
	Ordinal0 [0x00AE86E6+952038]
	GetHandleVerifier [0x00EC0CB2+2738370]
	GetHandleVerifier [0x00EB21B8+2678216]
	GetHandleVerifier [0x00CA17AA+512954]
	GetHandleVerifier [0x00CA0856+509030]
	Ordinal0 [0x00BB743B+1799227]
	Ordinal0 [0x00BBBB68+1817448]
	Ordinal0 [0x00BBBC55+1817685]
	Ordinal0 [0x00BC5230+1856048]
	BaseThreadInitThunk [0x75B8FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77B275F4+228]